In [1]:
import numpy as np
import cv2
import pandas as pd

# Homography Practice

1. Given image coordinate pairs $\( \mathbf{u}_1'=(u_1', v_1') \)$ and $\( \mathbf{u}_2'=(u_2', v_2') \)$, show a method that derives the homography matrix $H$ such that $\( \mathbf{u}_1' \approx H \mathbf{u}_2' \)$.

     1.a Derive a matrix A such that $A\mathbf{h} = 0$.
    
     1.b Derive a method for optimization.

2. Method of calculating the normalization matrix $T_{3x3}$ - Given point pairs $\( \mathbf{u}_1 \)$, find $T$.

3. Given $T_1$, $H$, $T_2$ derive a method to denormalize $H$.


# Solution for 1.a

- The equation $A\mathbf{h} = 0$ corresponds to the homography equation $\omega \mathbf{u} = H\mathbf{x}$, where:

$$
\omega
\begin{bmatrix}
u \\
v \\
1
\end{bmatrix}
=
\begin{bmatrix}
h_{11} & h_{12} & h_{13} \\
h_{21} & h_{22} & h_{23} \\
h_{31} & h_{32} & h_{33} \\
\end{bmatrix}
\begin{bmatrix}
x \\
y \\
1
\end{bmatrix}
$$

- Solving for $\omega$, we get:

$$
\omega = h_{31}x + h_{32}y + h_{33}
$$

- And for $\omega u$:

$$
\omega u = h_{11}x + h_{12}y + h_{13}
$$

- By substituting $\omega$ from the third equation into the second, we can express $u$ in terms of $x$ and $y$:

$$
u = \frac{h_{11}x + h_{12}y + h_{13}}{h_{31}x + h_{32}y + h_{33}}
$$

- This leads us to the following system of equations for each point correspondence:

$$
\begin{bmatrix}
x & y & 1 & 0 & 0 & 0 & -ux & -uy & -u \\
0 & 0 & 0 & x & y & 1 & -vx & -vy & -v \\
\end{bmatrix}
\begin{bmatrix}
h_{11} \\
h_{12} \\
h_{13} \\
h_{21} \\
h_{22} \\
h_{23} \\
h_{31} \\
h_{32} \\
h_{33} \\
\end{bmatrix}
= 0
$$

- The rank of matrix $A$ has to be at least 8. We need at least 4 correspondences to solve for $H$.


# Solution for 1.b
To approach the problem of solving the equation $A\mathbf{h} = 0$ while avoiding the trivial solution $\mathbf{h} = 0$, we use a constrained least squares (LSQ) method. This involves imposing a constraint on the norm of $\mathbf{h}$, specifically:

$$ \|\mathbf{h}\|^2 = 1 \text{, which can be rewritten as } \mathbf{h}^T\mathbf{h} = 1. $$

With this constraint, our objective becomes minimizing a cost function $L(\mathbf{h}, \lambda)$ defined as:

$$ L(\mathbf{h}, \lambda) = \|\mathbf{Ah}\|^2 + \lambda (\mathbf{h}^T\mathbf{h} - 1). $$

To find the minimum of this function, we first set its derivative with respect to $\mathbf{h}$ to zero:

$$ \frac{\partial L(\mathbf{h}, \lambda)}{\partial \mathbf{h}} = 0. $$

This leads us to the equation:

$$ \mathbf{A}^T\mathbf{Ah} - \lambda \mathbf{h} = 0. $$

Also, the derivative of $L$ with respect to $\lambda$ gives us the constraint equation:

$$ \lambda (\mathbf{h}^T\mathbf{h} - 1) = 0, $$

which simplifies to:

$$ \mathbf{h}^T\mathbf{h} = 1. $$

The key equation $\mathbf{A}^T\mathbf{Ah} = \lambda \mathbf{h}$ suggests that $\mathbf{h}$ is an eigenvector of $\mathbf{A}^T\mathbf{A}$ and $\lambda$ is the corresponding eigenvalue. Among all the eigenvalues of $\mathbf{A}^T\mathbf{A}$, the one that minimizes our problem is the smallest eigenvalue, $\lambda_{\min}$. 

Therefore, the solution $\mathbf{h}$ to our original problem is the eigenvector of $\mathbf{A}^T\mathbf{A}$ associated with its smallest eigenvalue $\lambda_{\min}$. This eigenvalue satisfies:

$$ \mathbf{h}^T\mathbf{A}^T\mathbf{Ah} = \lambda_{\min} \mathbf{h}^T\mathbf{h} = \lambda_{\min}. $$


## Task 2
# Derivation of the Normalization Matrix

The normalization matrix $T$ is used to scale and translate a set of points so that their centroid is at the origin and their average distance from the origin is $\sqrt{2}$, which enhances numerical stability for further calculations. The derivation of this matrix involves the following steps:

1. **Scaling**:
   - To scale the points, we multiply each coordinate by a scaling factor $s$. For a point $(u, v)$, this is represented as $s \cdot u$ and $s \cdot v$.

2. **Translation**:
   - After scaling, the points are translated such that the centroid is at the origin. For the scaled point $(u', v')$ and the original mean $(\bar{u}, \bar{v})$, the translated coordinates are $u'' = u' - \bar{u}$ and $v'' = v' - \bar{v}$.

3. **Combining Operations**:
   - The normalized coordinates $(u'', v'')$ can be expressed as:
     $$ u'' = s \cdot u - s \cdot \bar{u} $$
     $$ v'' = s \cdot v - s \cdot \bar{v} $$

4. **Matrix Form**:
   - We can express these operations in matrix form for homogeneous coordinates:
     $$ \begin{bmatrix} u'' \\ v'' \\ 1 \end{bmatrix} = \begin{bmatrix} s & 0 & -s \cdot \bar{u} \\ 0 & s & -s \cdot \bar{v} \\ 0 & 0 & 1 \end{bmatrix} \cdot \begin{bmatrix} u \\ v \\ 1 \end{bmatrix} $$

5. **Deriving Scale Factor $s$**:
   - The scale factor $s$ is determined by the condition that the average distance from the origin should be $\sqrt{2}$. If $\sigma$ is the average standard deviation from the centroid, then $s = \frac{\sqrt{2}}{\sigma}$.

6. **Final Matrix $T$**:
   - The normalization matrix $T$ is thus defined as:
     $$ T = \begin{bmatrix} \frac{\sqrt{2}}{\sigma} & 0 & -\frac{\sqrt{2}}{\sigma} \cdot \bar{u} \\ 0 & \frac{\sqrt{2}}{\sigma} & -\frac{\sqrt{2}}{\sigma} \cdot \bar{v} \\ 0 & 0 & 1 \end{bmatrix} $$
   - Here $\sigma$ is the average of the standard deviations of $u$ and $v$ coordinates: $\sigma = \frac{\text{std}(u) + \text{std}(v)}{2}$.

This matrix, when applied to a set of points, normalizes them for a mean of zero and an average distance from the origin of $\sqrt{2}$. This process is crucial for maintaining numerical stability in algorithms that compute transformations such as homography.



### Task 3

# Derivation of Denormalization for Homography Matrix

Given normalization matrices $T_1$ and $T_2$, and a normalized homography matrix $H$, we can derive a method to denormalize $H$. The denormalization process involves reversing the normalization transformations that were applied to the point sets before computing $H$.

## Normalization Process:
Points in the original space are transformed to the normalized space using $T_1$:

$$
\mathbf{u} = T_1 \mathbf{u}'
$$

## Applying the Homography:
The homography $H$ relates the normalized points in the first image to the normalized points in the second image:

$$
\mathbf{v} \sim H \mathbf{u}
$$

However, to express this relationship in terms of the original image coordinates, we need to apply the inverse of the second normalization matrix $T_2$:

$$
\mathbf{v}' = T_2^{-1} \mathbf{v}
$$

## Denormalization of Homography:
By substituting $\mathbf{u}$ and $\mathbf{v}$ into the homography equation, we can derive the denormalized homography matrix $H'$ that maps the original image coordinates:

$$
T_2^{-1} \mathbf{v} \sim T_2^{-1} H T_1 \mathbf{u}'
$$

Hence, the denormalized homography matrix $H'$ is obtained by:

$$
H' = T_2^{-1} H T_1
$$

This equation allows us to transform the normalized homography matrix back into the coordinate system of the original images.


In [2]:
# Function to calculate the normalization matrix
def calculate_normalization_matrix(points):
    mean = np.mean(points, axis=0)
    std = np.std(points, axis=0)
    std_avg = np.mean(std)
    scale = np.sqrt(2) / std_avg
    offset = -scale * mean
    T = np.array([[scale, 0, offset[0]],
                  [0, scale, offset[1]],
                  [0, 0, 1]])
    return T

# Function to apply the normalization matrix to the points
def normalize_points(points, T):
    normalized_points = []
    for point in points:
        point_homogeneous = np.append(point, 1)
        normalized_point = T @ point_homogeneous
        normalized_points.append(normalized_point[:2] / normalized_point[2])
    return np.array(normalized_points)

# Function to calculate the homography matrix
def calculate_homography(point_pairs):
    A = []
    for point1, point2 in point_pairs:
        x, y = point1[0], point1[1]
        xp, yp = point2[0], point2[1]
        A.append([-x, -y, -1, 0, 0, 0, x*xp, y*xp, xp])
        A.append([0, 0, 0, -x, -y, -1, x*yp, y*yp, yp])
    A = np.array(A)
    
    # U, S, Vt = np.linalg.svd(A)
    
    eig_val, eig_vec = np.linalg.eig(A.T@A)
    
    
    Vt = eig_vec[:,np.argmin(eig_val)]
    
    H = Vt.reshape(3, 3)
    
    return H / H[2, 2]

In [3]:
# Read the point pairs
point_pairs = np.genfromtxt('features.txt').T
pd.DataFrame(point_pairs, columns=['x1','y1','x2','y2'])

,x1,y1,x2,y2
0,211.0,212.0,439.0,213.0
1,174.0,335.0,400.0,340.0
2,182.0,360.0,408.0,366.0
3,54.0,304.0,281.0,303.0
4,88.0,313.0,311.0,313.0
5,24.0,206.0,254.0,208.0
6,140.0,114.0,362.0,114.0
7,37.0,105.0,261.0,110.0


In [4]:
points1 = point_pairs[:,:2]
points2 = point_pairs[:,-2:]

In [ ]:
# Normalize the points and calculate the homography matrix
T1 = calculate_normalization_matrix(points1)
T2 = calculate_normalization_matrix(points2)

normalized_points1 = normalize_points(points1, T1)
normalized_points2 = normalize_points(points2, T2)

normalized_point_pairs = list(zip(points1, points2))
H = calculate_homography(normalized_point_pairs)

# Denormalize the homography matrix
#H = np.linalg.inv(T2) @ H_normalized @ T1

# Read the images
img1 = cv2.imread('b.png')
img2 = cv2.imread('a.png')

# Warp the second image to the first using the homography matrix
img2_transformed = cv2.warpPerspective(img2, H, (img1.shape[1] + img2.shape[1], img1.shape[0]))

# Stitch the images together
stitched_img = img2_transformed
stitched_img[0:img1.shape[0], 0:img1.shape[1]] = img1

# Save the stitched image
cv2.imwrite('stitched_image.png', stitched_img)

# Display the stitched image
cv2.imshow('Stitched Image', stitched_img)
cv2.waitKey(0)
cv2.destroyAllWindows()
